# Resumen
- missing_data : como esta
- localizacion: no merece la pena hacer tabla dimensiones
- amounts in fares

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder \
    .appName("SmallTestDataset") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
#INPUT DATA
input_data_g = "/home/jupyter/proyect/ds_taxi_NY/green_tripdata_2024-01.parquet"
input_data_y = "/home/jupyter/proyect/ds_taxi_NY/yellow_tripdata_2024-01.parquet"

In [4]:
#READ FILE GREEN TAXI
df_g = spark.read.parquet(input_data_g)

#SELECCION VARIABLES GREEN DATASET
lista_vars = ['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfg = df_g.select(lista_vars)

# TRANSFORMACION REDONDEO TIEMPO, Y DATE GREEN TAXI
dfg = dfg.withColumn("date_init_trip", F.date_format(F.col("lpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("lpep_pickup_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("lpep_dropoff_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("lpep_dropoff_datetime") )) \
         .drop(F.col('lpep_pickup_datetime'))\
         .drop( F.col('lpep_dropoff_datetime') )

In [27]:
input_data_y = "/home/jupyter/proyect/ds_taxi_NY/yellow_tripdata_2024-01.parquet"
df_y = spark.read.parquet(input_data_y)

#SELECCION VARIABLES YELLOW DATASET
lista_vars = ['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfy = df_y.select(lista_vars)


# TRANSFORMACION REDONDEO TIEMPO, Y DATE YELLOW TAXI
dfy = dfy.withColumn("date_init_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("tpep_dropoff_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("tpep_dropoff_datetime") )) \
         .drop(F.col('tpep_pickup_datetime'))\
         .drop( F.col('tpep_dropoff_datetime') )

In [ ]:
#df = dfg.union(dfy)

## MISSING DATA
- por ahora dejo los datos tal cuales

In [13]:
df_g.select([F.count(  F.when(F.col(c).isNull(),c) ).alias(f'{c}_Null_count') for c in df_g.columns]).show()

+-------------------+-------------------------------+--------------------------------+-----------------------------+---------------------+-----------------------+-----------------------+--------------------------+------------------------+----------------------+----------------+------------------+---------------------+-----------------------+--------------------+--------------------------------+-----------------------+-----------------------+--------------------+-------------------------------+
|VendorID_Null_count|lpep_pickup_datetime_Null_count|lpep_dropoff_datetime_Null_count|store_and_fwd_flag_Null_count|RatecodeID_Null_count|PULocationID_Null_count|DOLocationID_Null_count|passenger_count_Null_count|trip_distance_Null_count|fare_amount_Null_count|extra_Null_count|mta_tax_Null_count|tip_amount_Null_count|tolls_amount_Null_count|ehail_fee_Null_count|improvement_surcharge_Null_count|total_amount_Null_count|payment_type_Null_count|trip_type_Null_count|congestion_surcharge_Null_count|
+-

In [21]:
dfx = df_g.filter(F.col('store_and_fwd_flag').isNull() )
print( dfx.count() ) 
dfx.select( F.count(F.col('passenger_count') ) ).show()

3415
+----------------------+
|count(passenger_count)|
+----------------------+
|                     0|
+----------------------+



In [24]:
dfx.select( F.min(F.col('store_and_fwd_flag') ) ).show()

+-----------------------+
|min(store_and_fwd_flag)|
+-----------------------+
|                   null|
+-----------------------+



In [25]:
df_g.filter(F.col('store_and_fwd_flag').isNotNull() ).select(F.col('store_and_fwd_flag')).distinct().show()

+------------------+
|store_and_fwd_flag|
+------------------+
|                 Y|
|                 N|
+------------------+



In [26]:
df_g.filter(F.col('store_and_fwd_flag').isNotNull() ).groupBy('store_and_fwd_flag').count().show()

+------------------+-----+
|store_and_fwd_flag|count|
+------------------+-----+
|                 Y|  115|
|                 N|53021|
+------------------+-----+



In [7]:
df_g.show(20)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2024-01-01 00:46:55|  2024-01-01 00:58:25|                 N|         1|         236|         239|              1|         1.98|       12.8|  1.0|    0.

## LOCATIONS

In [30]:
dfy.count()

2964624

In [ ]:
dfy.groupBy( ['PULocationID', 'DOLocationID']).count().orderBy(F.col('count').desc()).show(50)

+------------+------------+-----+
|PULocationID|DOLocationID|count|
+------------+------------+-----+
|         237|         236|21883|
|         236|         237|19402|
|         236|         236|15955|
|         237|         237|14938|
|         161|         237|10275|
|         142|         239| 8980|
|         237|         161| 8834|
|         161|         236| 8766|
|         239|         142| 8675|
|         239|         238| 8445|
|         141|         236| 7803|
|         237|         162| 7604|
|         264|         264| 7381|
|         132|         132| 7252|
|         236|         161| 7115|
|         186|         230| 7085|
|         142|         238| 7073|
|         238|         239| 7007|
|         236|         141| 6930|
|         236|         239| 6808|
|         237|         141| 6668|
|         263|         236| 6574|
|         236|         238| 6385|
|         263|         141| 6303|
|         162|         237| 6301|
|         239|         236| 6135|
|         140|

## Amounts
comprobar si las cantidades son iguales

In [40]:
df_g.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [44]:
 lvar = [ 'RatecodeID','fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'ehail_fee',
            'improvement_surcharge',
            'total_amount']
df_g_temp = df_g.select(lvar)
df_g_temp.show()

+----------+-----------+-----+-------+----------+------------+---------+---------------------+------------+
|RatecodeID|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|
+----------+-----------+-----+-------+----------+------------+---------+---------------------+------------+
|         1|       12.8|  1.0|    0.5|      3.61|         0.0|     null|                  1.0|       21.66|
|         1|       30.3|  1.0|    0.5|      7.11|         0.0|     null|                  1.0|       42.66|
|         1|       19.8|  1.0|    0.5|       3.0|         0.0|     null|                  1.0|       28.05|
|         1|       14.2|  1.0|    1.5|       0.0|         0.0|     null|                  1.0|        16.7|
|         1|       22.6|  1.0|    0.5|      6.28|         0.0|     null|                  1.0|       31.38|
|         1|       17.0| 3.75|    1.5|       2.0|         0.0|     null|                  1.0|       24.25|
|         1|       18.4| 3.7

In [45]:
df_g_temp.withColumn('total_amount', F.col('total_amount')+F.col('tip_amount'))

DataFrame[RatecodeID: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: double, improvement_surcharge: double, total_amount: double]